# Numerical PDE III: Spectral-Galerkin Method

In fluid dynamics, spectral-Galerkin methods are commonly used to
solve the incompressible Navier-Stokes equations.
By transforming the equations into spectral space, the nonlinear terms
can be computed efficiently while maintaining conservation properties;
and the viscous term can be easily solve implicitly.
For incompressible flows, the vorticity-streamfunction formulation is
particularly usable, as it eliminates the pressure term and reduces
the computational complexity.

In this lecture, we will focus on applying spectral methods to solve
the 2D incompressible hydrodynamics equations.
Specifically, we will:
1. Derive the governing equations in spectral space.
2. Discuss the vorticity-streamfunction formulation and its
   advantages.
3. Implement a spectral solver to simulate the evolution of vorticity
   in a 2D periodic domain.

This introduction sets the stage for understanding how spectral
methods leverage mathematical elegance and computational efficiency to
solve complex PDEs with remarkable accuracy.